In [13]:
#import dependencies
import pandas as pd
from sqlalchemy import create_engine

In [14]:
#Import the CSV file from the first data source and extract it into DataFrame
vehicle_file = "Resources/vehicles-2.csv"
vehicle_df = pd.read_csv(vehicle_file)
vehicle_df.head()

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,...,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0


In [15]:
#Import the CSV file from the second data source and extract it into DataFrame
used_car_file = "Resources/true_car_listings.csv"
used_car_df = pd.read_csv(used_car_file)
used_car_df.head()

,Price,Year,Mileage,City,State,Vin,Make,Model
0,8995,2014,35725,El Paso,TX,19VDE2E53EE000083,Acura,ILX6-Speed
1,10888,2013,19606,Long Island City,NY,19VDE1F52DE012636,Acura,ILX5-Speed
2,8995,2013,48851,El Paso,TX,19VDE2E52DE000025,Acura,ILX6-Speed
3,10999,2014,39922,Windsor,CO,19VDE1F71EE003817,Acura,ILX5-Speed
4,14799,2016,22142,Lindon,UT,19UDE2F32GA001284,Acura,ILXAutomatic


In [16]:
#Begin the transformation process by creatign a subset data frame with ONLY the desired coulmns.
vehicle_col = ['make', 'model', 'year','co2TailpipeGpm','fuelType', 'fuelType1']
vehicle_transformation_df = vehicle_df[vehicle_col]
vehicle_transformation_df.head()

,make,model,year,co2TailpipeGpm,fuelType,fuelType1
0,Alfa Romeo,Spider Veloce 2000,1985,423.190476,Regular,Regular Gasoline
1,Ferrari,Testarossa,1985,807.909091,Regular,Regular Gasoline
2,Dodge,Charger,1985,329.148148,Regular,Regular Gasoline
3,Dodge,B150/B250 Wagon 2WD,1985,807.909091,Regular,Regular Gasoline
4,Subaru,Legacy AWD Turbo,1993,467.736842,Premium,Premium Gasoline


In [17]:
#rename the dataframe columns to make them more SQL friendly
vehicle_transformed = vehicle_transformation_df.rename(columns={
   'co2TailpipeGpm': 'co2_tailpipe_gpm',
    'fuelType': 'fuel_type',
    'fuelType1':'fuel_type1'
})
vehicle_transformed.head()

,make,model,year,co2_tailpipe_gpm,fuel_type,fuel_type1
0,Alfa Romeo,Spider Veloce 2000,1985,423.190476,Regular,Regular Gasoline
1,Ferrari,Testarossa,1985,807.909091,Regular,Regular Gasoline
2,Dodge,Charger,1985,329.148148,Regular,Regular Gasoline
3,Dodge,B150/B250 Wagon 2WD,1985,807.909091,Regular,Regular Gasoline
4,Subaru,Legacy AWD Turbo,1993,467.736842,Premium,Premium Gasoline


In [18]:
#add a distinct id coumn that would map to the id column of its respective SQL table 
vehicle_transformed['id'] = vehicle_transformed.index
vehicle_transformed.head()

,make,model,year,co2_tailpipe_gpm,fuel_type,fuel_type1,id
0,Alfa Romeo,Spider Veloce 2000,1985,423.190476,Regular,Regular Gasoline,0
1,Ferrari,Testarossa,1985,807.909091,Regular,Regular Gasoline,1
2,Dodge,Charger,1985,329.148148,Regular,Regular Gasoline,2
3,Dodge,B150/B250 Wagon 2WD,1985,807.909091,Regular,Regular Gasoline,3
4,Subaru,Legacy AWD Turbo,1993,467.736842,Premium,Premium Gasoline,4


In [19]:
#Similarly, repeat the transformation process for the data frame from the second source by 
#creating a subset data frame with ONLY the desired coulmns.
used_car_col = ['Year', 'Make', 'Model','Price','Mileage','City','State','Vin']
used_car_transformation_df = used_car_df[used_car_col]

In [20]:
#rename the dataframe columns to make them more SQL friendly
used_car_transformed = used_car_transformation_df.rename(columns={
    'Year': 'year',
    'Make': 'make',
    'Model': 'model',
    'Price':'price',
    'Mileage':'mileage',
    'City':'city',
    'State':'state',
    'Vin':'vin'
})

In [21]:
used_car_transformed.head()

,year,make,model,price,mileage,city,state,vin
0,2014,Acura,ILX6-Speed,8995,35725,El Paso,TX,19VDE2E53EE000083
1,2013,Acura,ILX5-Speed,10888,19606,Long Island City,NY,19VDE1F52DE012636
2,2013,Acura,ILX6-Speed,8995,48851,El Paso,TX,19VDE2E52DE000025
3,2014,Acura,ILX5-Speed,10999,39922,Windsor,CO,19VDE1F71EE003817
4,2016,Acura,ILXAutomatic,14799,22142,Lindon,UT,19UDE2F32GA001284


In [22]:
#Add a unique id column
used_car_transformed=used_car_transformed.reset_index(drop=False)
used_car_transformed.head()

,index,year,make,model,price,mileage,city,state,vin
0,0,2014,Acura,ILX6-Speed,8995,35725,El Paso,TX,19VDE2E53EE000083
1,1,2013,Acura,ILX5-Speed,10888,19606,Long Island City,NY,19VDE1F52DE012636
2,2,2013,Acura,ILX6-Speed,8995,48851,El Paso,TX,19VDE2E52DE000025
3,3,2014,Acura,ILX5-Speed,10999,39922,Windsor,CO,19VDE1F71EE003817
4,4,2016,Acura,ILXAutomatic,14799,22142,Lindon,UT,19UDE2F32GA001284


In [23]:
#rename the index column to id as it would map to the id column of its respective SQL table 
used_car_transformed2 = used_car_transformed.rename(columns={'index': 'id'})
used_car_transformed2.head()

,id,year,make,model,price,mileage,city,state,vin
0,0,2014,Acura,ILX6-Speed,8995,35725,El Paso,TX,19VDE2E53EE000083
1,1,2013,Acura,ILX5-Speed,10888,19606,Long Island City,NY,19VDE1F52DE012636
2,2,2013,Acura,ILX6-Speed,8995,48851,El Paso,TX,19VDE2E52DE000025
3,3,2014,Acura,ILX5-Speed,10999,39922,Windsor,CO,19VDE1F71EE003817
4,4,2016,Acura,ILXAutomatic,14799,22142,Lindon,UT,19UDE2F32GA001284


In [34]:
###############################################################################################################
#Go to PostgreSQL and create the database automobile_db, two tables for each of the two data frames, 
#and create the column names for each table exactly as they were renamed for the SQL convention
#in the above data frames.
###############################################################################################################

In [24]:
#create a connection pandas and to the SQL environment - in this case PostgreSQL
rds_connection_string = "postgres:NIM2006@localhost:5432/automobile_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [25]:
#Once the tables are created on the PostgreSQL side, verify the connection is working by recalling 
#the table names that were created in PostGreSQL inside the automobile_db database.
engine.table_names()

['emission', 'used_car']

In [34]:
#Load the information from the dataframe to its respective table in postgreSQL
vehicle_transformed.to_sql(name='emission', con=engine, if_exists='append', index=False)

In [35]:
#Repeat the above for the second dataframe by Loading the information from the dataframe to its 
#respective table in postgreSQL
used_car_transformed2.to_sql(name='used_car', con=engine, if_exists='append', index=False)

In [29]:
!jupyter nbconvert --to script used_car_emissions.ipynb

[NbConvertApp] Converting notebook used_car_emissions.ipynb to script
[NbConvertApp] Writing 2106 bytes to used_car_emissions.py
